# Midterm

In [10]:
import pandas as pd
from statsmodels.sandbox.regression.gmm import GMM

## Part 1

In [11]:
df = pd.read_csv(r"C:\Users\johns\OneDrive\Desktop\MBAN Semester 2\Predictive Analytics\Midterm\midterm_partone.csv")

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1696 entries, 0 to 1695
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Constant            1696 non-null   int64  
 1   Stock Change        1696 non-null   float64
 2   Inventory Turnover  1696 non-null   float64
 3   Operating Profit    1696 non-null   float64
 4   Interaction Effect  1696 non-null   float64
 5   Current Ratio       1696 non-null   float64
 6   Quick Ratio         1696 non-null   float64
 7   Debt Asset Ratio    1696 non-null   float64
dtypes: float64(7), int64(1)
memory usage: 106.1 KB


In [13]:
df.head()

,Constant,Stock Change,Inventory Turnover,Operating Profit,Interaction Effect,Current Ratio,Quick Ratio,Debt Asset Ratio
0,1,0.870332,1.795946,0.115846,0.208053,1.672527,0.255171,0.473317
1,1,-0.047347,1.395501,0.436967,0.609788,1.637261,0.221763,0.489967
2,1,0.001176,1.664563,0.541016,0.900555,1.640619,0.189141,0.374269
3,1,-0.901200,1.605738,0.539399,0.866133,1.436221,0.131944,0.224399
4,1,-0.176353,1.591451,0.539938,0.859285,1.433140,0.183095,0.213446


In [21]:
y_vals  = np.array(df["Stock Change"])                                                  # Endogenous variable
x_vals  = np.array(df[["Inventory Turnover","Operating Profit","Interaction Effect"]])  # Exogenous variables
iv_vals = np.array(df[["Current Ratio","Quick Ratio","Debt Asset Ratio"]])              # Instrumental variables

class gmm(GMM):
    def momcond(self, params):
        p0, p1, p2, p3 = params
        endog = self.endog
        exog = self.exog
        inst = self.instrument   

        error0 =  endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]
        error1 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * exog[:,1]
        error2 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * exog[:,2]
        error3 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,0] 
        error4 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,1] 
        error5 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,2] 

        bias_term = np.array([[1, 1, 1]] * len(endog)) * self.delta
        g = np.column_stack((error0, error1, error2, error3, error4, error5, bias_term))
        return g

gmm_model = gmm(endog=y_vals, exog=x_vals, instrument=iv_vals, k_moms=7, k_params=4)

gmm_model.delta = 0.1

beta0 = np.array([1, 1, 1, 1])
res = gmm_model.fit(beta0)

res.summary()

ValueError: shapes (9,) and (7,7) not aligned: 9 (dim 0) != 7 (dim 0)